In [14]:
import numpy as np
import sympy as sp


def function(x, y):
    '''Define the polynomial function f(x, y).'''
    return x**2 + y**2 - 1    # (x**3 - x *y**2 + y + 1)**2  (x**2 + y**2 - 1) + y**2 - 5

def gradient(f):
    '''Calculate the symbolic gradient of the function f.'''
    x, y = sp.symbols('x y')
    f_sym = f(x, y)
    df_dx = sp.diff(f_sym, x)
    df_dy = sp.diff(f_sym, y)
    return sp.lambdify((x, y), df_dx), sp.lambdify((x, y), df_dy)

def grid(x0, y0, B, Num_base_points):
    '''
    Return two 2-dimensional arrays representing the X and Y coordinates
    of all points in a regular square grid centered at (x0, y0),
    with side length 2B and Num_base_points^2 points.
    '''
    x = np.linspace(x0 - B, x0 + B, Num_base_points)
    y = np.linspace(y0 - B, y0 + B, Num_base_points)
    return np.meshgrid(x, y, indexing='xy')

def norm_f_on_grid(grid):
    '''
    Return a 2-dimensional array containing the absolute value of
    the function evaluated at the points in the grid.
    '''
    return np.absolute(function(grid[0], grid[1]))

def L1norm_grad_on_grid(grid):
    '''
    Return a 2-dimensional array containing the L1-norm of the gradient
    of the function evaluated at the points in the grid.
    '''
    grad_f = gradient(function)
    return np.absolute(grad_f[0](grid[0], grid[1])) + np.absolute(grad_f[1](grid[0], grid[1]))



# # # # # # # # # # # # # # # 
# Generate the grid on the bounding box
Num_base_points = 100  # number of base points, SHOULD BE AT LEAST 3
B = 2 # bounding box side length
init_point = np.array([0, 0])
X, Y = grid(init_point[0], init_point[1], B, Num_base_points)
# # # # # # # # # # # # # # # # 



def L2norm_grad_on_grid(grid):
    '''
    Return a 2-dimensional array containing the L2-norm (Euclidean norm) 
    of the gradient of the function evaluated at the points in the grid.
    '''
    grad_f = gradient(function)
    grad_x = grad_f[0](grid[0], grid[1])
    grad_y = grad_f[1](grid[0], grid[1])
    return np.sqrt(grad_x**2 + grad_y**2)

def hessian(f):
    '''Calculate the symbolic Hessian matrix of the function f using SymPy.'''
    x, y = sp.symbols('x y')
    f_sym = f(x, y)
    hessian_matrix = sp.hessian(f_sym, (x, y))
    return sp.lambdify((x, y), hessian_matrix)
    
def hessian_L21_norm(hessian_func, X, Y):
    '''
    Compute the L_{21}-norm (sum of Euclidean norms of rows) of the Hessian
    at each point on the grid, returning the maximum value.
    '''
    L21_norms = np.zeros_like(X)

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            H = np.array(hessian_func(X[i, j], Y[i, j]), dtype=np.float64)
            row_norms = np.sqrt(np.sum(H**2, axis=1))  # Euclidean norm of each row
            L21_norms[i, j] = np.sum(row_norms)

    return L21_norms

def largest_eigenvalue(hessian_func, X, Y):
    '''Computes the largest eigenvalue of the Hessian at each point on the grid.'''
    largest_eigenvalues = np.zeros_like(X)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            hessian_matrix = np.array(hessian_func(X[i, j], Y[i, j]), dtype=np.float64)
            eigenvalues = np.linalg.eigvals(hessian_matrix)
            largest_eigenvalues[i, j] = np.max(eigenvalues)
    
    return largest_eigenvalues
    
# Compute M3 using the L21-norm of the Hessian
M3 = np.max(hessian_L21_norm(hessian(function), X, Y) )
print("Upper bound of L21-norm of the Hessian (M3):", M3)

# Compute M2 using the L2-norm of the gradient
M2 = np.max(L2norm_grad_on_grid([X, Y]))
print("Upper bound of the gradient L2-norm (M2):", M2)

# Compute K using the L2-norm of the Hessian (not needed in the algorithm)
#K = np.max(largest_eigenvalue(hessian(function), X, Y) )
#print("Upper bound of L2-norm of the Hessian (K):", K)

def dubious_points(mesh, a_tolerance):
    '''
    Given a grid and an absolute tolerance (epsilon), return a list of points
    for which both the absolute value of the function and its gradient norm
    are smaller than thresholds based on the given tolerance.
    '''
    threshold_f = np.sqrt(2) * a_tolerance * M2         # $|f(m)|> \sqrt{N} \epsilon M_2$
    threshold_grad = 2 * np.sqrt(2) * a_tolerance * M3  # $|\nabla f(m)|_1> N^{3/2} \epsilon M_3 $
    relative_coords = np.where(
        np.isclose(norm_f_on_grid(mesh), 0, atol=threshold_f)
        & np.isclose(L1norm_grad_on_grid(mesh), 0, atol=threshold_grad)
    )
    return np.column_stack((mesh[0][relative_coords], mesh[1][relative_coords])).tolist()

def bound_df(init_point, B, Num_base_points, epsilon_min=1e-5):
    '''
    Iteratively refine the search for points where the function and its gradient
    are close to zero, adjusting epsilon and refining the grid around dubious points.
    '''
    epsilon = 2 * B / (Num_base_points - 1)
    final_dubious_points = []

    while epsilon > epsilon_min:
        mesh = grid(init_point[0], init_point[1], B, Num_base_points)
        list_dub_points = dubious_points(mesh, epsilon)

        #print(f"epsilon={epsilon}")
        #print(f"Coords of dubious points={list_dub_points}\n")

        if not list_dub_points:
            print("No dubious points found, stopping at epsilon:", epsilon)
            break
        else:
            final_dubious_points = list_dub_points
            B = epsilon
            epsilon = epsilon / (Num_base_points - 1)

            refined_dubious_points = []
            for coordinate in list_dub_points:
                new_mesh = grid(coordinate[0], coordinate[1], B, Num_base_points)
                new_dub_points = dubious_points(new_mesh, epsilon)
                refined_dubious_points.extend(new_dub_points)

            final_dubious_points = refined_dubious_points

    return  np.sqrt(2) * epsilon * M3, final_dubious_points    #prop 3.1 using N=2

# Example 
final_points = bound_df(init_point, B=2, Num_base_points=3, epsilon_min=1e-5)
print("Bound on |df|_1 and final dubious points found:", final_points)


Upper bound of L21-norm of the Hessian (M3): 4.0
Upper bound of the gradient L2-norm (M2): 5.656854249492381
No dubious points found, stopping at epsilon: 0.0625
Bound on |df|_1 and final dubious points found: (0.3535533905932738, [])
